In [1]:
# coding: utf-8
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')



In [16]:
top="33173 33475 1076 35300 15743"

def test_item_filter(x):
    for i in x[1]:
        if i[0] in test_users:
            return True
    return False

def map_string (x):
        y=x[1]
        s=""
        for i in y:
            s+=str(i[0])+" "
        z=len(y)
        top_rates=[33173, 33475, 1076, 35300, 15743]
        i=0
        while (z<5):
            s+=str(top_rates[i])
            i+=1
            z+=1
        if (y[0][1]<8):
            s=top
        return x[0],s

#feature1, feature2: [[feature_1],[feature_2],...[feature_N]]
def jaccard_sim(feature1,feature2):
    s1=set([feat[0] for feat in feature1])
    s2=set([feat[0] for feat in feature2])
    if len(s1 | s2)==0: return 0
    return float(len(s1 & s2)) / len(s1 | s2)

feat_rdd=sc.textFile('icm_no_header.csv')
train_rdd = sc.textFile("train_no_header.csv")
test_rdd = sc.textFile("test_no_header.csv")
test_users=test_rdd.collect()[0:10]




In [17]:
feat_rdd=feat_rdd.map(lambda x: x.split(',')).map(lambda x: (x[0],[x[1]]))

test_item_votes_rdd=(train_rdd
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])]))
                     .groupByKey()
                     .filter(test_item_filter)
                     .map(lambda x: (x[0],list(x[1]))))

test_items=test_item_votes_rdd.map(lambda x:x[0]).collect()
item_feat_rdd=feat_rdd.groupByKey()
item_feat_rdd.map(lambda x:(x[0],list(x[1]))).take(10)


[(u'35236', [[u'1573'], [u'5786'], [u'5787'], [u'9029'], [u'14755']]),
 (u'35540', [[u'18904']]),
 (u'35542', [[u'268'], [u'17043'], [u'17181'], [u'17805'], [u'18904']]),
 (u'3382', [[u'566'], [u'2277'], [u'9399'], [u'17952'], [u'18904']]),
 (u'35544',
  [[u'2403'], [u'3248'], [u'9003'], [u'9004'], [u'12851'], [u'19584']]),
 (u'13357',
  [[u'2068'],
   [u'2275'],
   [u'4557'],
   [u'5787'],
   [u'10869'],
   [u'15634'],
   [u'18904']]),
 (u'35546',
  [[u'3332'],
   [u'5787'],
   [u'9004'],
   [u'9727'],
   [u'12362'],
   [u'12543'],
   [u'18549']]),
 (u'35548',
  [[u'1536'], [u'5787'], [u'11455'], [u'15634'], [u'16260'], [u'16602']]),
 (u'24029',
  [[u'1829'], [u'4820'], [u'8922'], [u'12701'], [u'13169'], [u'15634']]),
 (u'32239', [[u'4804'], [u'5786'], [u'10801'], [u'13846']])]

In [18]:
#only the features for the item voted by the test users
test_item_feat_rdd=item_feat_rdd.filter(lambda x: x[0] in test_items)
test_item_feat_rdd.map(lambda x:(x[0], list(x[1]))).take(10)



[(u'4244',
  [[u'3828'], [u'4346'], [u'8922'], [u'9122'], [u'15200'], [u'18904']]),
 (u'20252', [[u'9879'], [u'10708'], [u'18463'], [u'18904']]),
 (u'29260',
  [[u'515'],
   [u'4638'],
   [u'7482'],
   [u'9003'],
   [u'9004'],
   [u'10444'],
   [u'12362']]),
 (u'11012', [[u'5787'], [u'12284'], [u'14260'], [u'17986']]),
 (u'33755',
  [[u'3232'],
   [u'4601'],
   [u'10779'],
   [u'12362'],
   [u'15634'],
   [u'16920'],
   [u'18904']]),
 (u'5861', [[u'621'], [u'2405'], [u'4715'], [u'7577'], [u'12284']]),
 (u'19036',
  [[u'303'], [u'3293'], [u'5470'], [u'10273'], [u'15634'], [u'18904']]),
 (u'32956',
  [[u'2483'], [u'4913'], [u'5787'], [u'6262'], [u'12361'], [u'14848']]),
 (u'13649',
  [[u'5786'],
   [u'6215'],
   [u'7412'],
   [u'10081'],
   [u'12362'],
   [u'15268'],
   [u'18904']]),
 (u'17924', [[u'845'], [u'5309'], [u'5960'], [u'18904']])]

In [38]:
#here the order in which we consider the cartesian product matters. Similarity of item1,item2 is equal to 
#similarity of item2,item1 but we must consider both the case in which the user has voted item1 and 
# the case in which user has voted item2. So, the cartesian is a total cartesian
item_feat_rdd_cart1 = item_feat_rdd.cartesian(item_feat_rdd)
item_feat_rdd_cart2 = item_feat_rdd_cart1.map(lambda x: (x[1],x[0]))

item_feat_rdd_cart = item_feat_rdd_cart1.union(item_feat_rdd_cart2)
item_feat_rdd_cart.take(100)

[((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e372d0>),
  (u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e37450>)),
 ((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e37c10>),
  (u'35540', <pyspark.resultiterable.ResultIterable at 0x7fef35e7cd10>)),
 ((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e37c10>),
  (u'35542', <pyspark.resultiterable.ResultIterable at 0x7fef35e7c910>)),
 ((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e7c810>),
  (u'3382', <pyspark.resultiterable.ResultIterable at 0x7fef35e7cc50>)),
 ((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e7c810>),
  (u'35544', <pyspark.resultiterable.ResultIterable at 0x7fef35e7cfd0>)),
 ((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e7c810>),
  (u'13357', <pyspark.resultiterable.ResultIterable at 0x7fef35e7c8d0>)),
 ((u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e7c810>),
  (u'35546', <pyspark.resultiterable.ResultIte

In [40]:
#keep only the tests users
test_item_feat_rdd_cart = (item_feat_rdd_cart
                           .filter(lambda x: (x[0][0] in test_items and x[1][0] not in test_items)))

print test_items
test_item_feat_rdd_cart.take(10)

[u'20252', u'29260', u'11012', u'19036', u'5861', u'14976', u'17924', u'23431', u'12750', u'3100', u'14716', u'11366', u'20959', u'10028', u'26454', u'36855', u'5681', u'36204', u'27604', u'13649', u'19841', u'3674', u'33755', u'19188', u'25912', u'4244', u'15917', u'27648', u'10820', u'22513', u'17782', u'9607', u'24063', u'23217', u'32019', u'32956', u'14629', u'16397', u'24479', u'10766', u'14669', u'4911', u'24440', u'19907', u'20428', u'30979', u'11042', u'14322', u'17107', u'17815', u'25656', u'14878', u'17701', u'3981', u'33213', u'24514', u'2164', u'31444', u'9855', u'16511']


[((u'4244', <pyspark.resultiterable.ResultIterable at 0x7fef35e37310>),
  (u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35e371d0>)),
 ((u'20252', <pyspark.resultiterable.ResultIterable at 0x7fef35e85390>),
  (u'35236', <pyspark.resultiterable.ResultIterable at 0x7fef35f0bed0>)),
 ((u'4244', <pyspark.resultiterable.ResultIterable at 0x7fef35f0bf50>),
  (u'35540', <pyspark.resultiterable.ResultIterable at 0x7fef35f0be50>)),
 ((u'4244', <pyspark.resultiterable.ResultIterable at 0x7fef35f0b150>),
  (u'35542', <pyspark.resultiterable.ResultIterable at 0x7fef35f0b190>)),
 ((u'20252', <pyspark.resultiterable.ResultIterable at 0x7fef35ef4e50>),
  (u'35540', <pyspark.resultiterable.ResultIterable at 0x7fef35ef4f10>)),
 ((u'20252', <pyspark.resultiterable.ResultIterable at 0x7fef35ef4e50>),
  (u'35542', <pyspark.resultiterable.ResultIterable at 0x7fef35f0b190>)),
 ((u'4244', <pyspark.resultiterable.ResultIterable at 0x7fef35ef4ad0>),
  (u'3382', <pyspark.resultiterable.ResultIterabl

In [69]:
#compute similarity between items
test_item_item_sim = (test_item_feat_rdd_cart
                      .map(lambda x: ([x[0][0],x[1][0]],jaccard_sim(x[0][1],x[1][1]))))

In [70]:
test_item_item_sim.take(10)

[([u'4244', u'35236'], 0.0),
 ([u'20252', u'35236'], 0.0),
 ([u'4244', u'35540'], 0.16666666666666666),
 ([u'4244', u'35542'], 0.1),
 ([u'20252', u'35540'], 0.25),
 ([u'20252', u'35542'], 0.125),
 ([u'4244', u'3382'], 0.1),
 ([u'4244', u'35544'], 0.0),
 ([u'4244', u'13357'], 0.08333333333333333),
 ([u'4244', u'35546'], 0.0)]

In [96]:
#shaping for join

test_item_votes_rdd = test_item_votes_rdd.map(lambda x: ((),[x[0],x[1][1]]))



test_item_user_vote=(train_rdd
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])])))
                     #.groupByKey()
                     #.map(lambda x: (x[0], list(x[1]))))
test_item_user_vote.take(10)

[(u'1', [u'2738', 1]),
 (u'1', [u'4716', 4]),
 (u'1', [u'13298', 8]),
 (u'1', [u'15122', 4]),
 (u'2', [u'11326', 5]),
 (u'3', [u'3406', 4]),
 (u'3', [u'3906', 10]),
 (u'3', [u'6345', 6]),
 (u'3', [u'7393', 8]),
 (u'4', [u'815', 8])]

In [71]:
#shaping for join
test_item_item_sim = test_item_item_sim.map(lambda x: (x[0][0],[x[0][1],x[1]]))
test_item_item_sim.take(10)


[(u'4244', [u'35236', 0.0]),
 (u'20252', [u'35236', 0.0]),
 (u'4244', [u'35540', 0.16666666666666666]),
 (u'4244', [u'35542', 0.1]),
 (u'20252', [u'35540', 0.25]),
 (u'20252', [u'35542', 0.125]),
 (u'4244', [u'3382', 0.1]),
 (u'4244', [u'35544', 0.0]),
 (u'4244', [u'13357', 0.08333333333333333]),
 (u'4244', [u'35546', 0.0])]

In [97]:
#
joined_item_sim = test_item_user_vote.join(test_item_item_sim)
joined_item_sim.take(10)

KeyboardInterrupt: 

In [76]:
def compute_rec(joined_item_sim_vote, item_sim):
    k=1   #change this in order to change the weight of a vote
    return user_vote[1]*item_sim[1]*k

user_item_rec = (joined_item_sim.map(lambda x: (x[1][0],[x[2][0],compute_rec(x[1],x[2])]))
                .groupByKey())
                #.map(lambda x: (x[0],sorted(list(y for y in x[1]),key=lambda x: -x[1]))))#.take(10)
    


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 144.0 failed 1 times, most recent failure: Lost task 3.0 in stage 144.0 (TID 153, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/nico/spark-1.5.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/nico/spark-1.5.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/nico/spark-1.5.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/nico/spark-1.5.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2355, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/nico/spark-1.5.1-bin-hadoop2.6/python/pysp